In [58]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [59]:
wiki_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [60]:
wiki_tool.name

'wikipedia'

In [61]:
import os
from dotenv import load_dotenv

dotenv_path = '../.env'  # Adjust the path to where your .env file is relative to your script
load_dotenv(dotenv_path)

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

In [62]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20).split_documents(docs)
vectordb=FAISS.from_documents(documents,AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    model="text-embedding-3-small",
    azure_deployment="pt-sm-embd"
    ))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x137bc2d90>)

In [63]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "search for information about langsmith. For any question about LangSmith you must use this tool")

In [64]:
retriever_tool.name

'langsmith_search'

In [65]:
## Arxiv where all the research papers have been uploaded
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [66]:
combined_tools=[wiki_tool, arxiv_tool,retriever_tool]

In [67]:
combined_tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/talwapul/Updated-Langchain/Updated-Langchain/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='search for information about langsmith. For any question about LangSmith you must use this tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x11304c680>, retriever=VectorStoreRetriever(tags=['FAISS', 'Azure

In [80]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    model_name="gpt-4",  # Specify the model name
    deployment_name="pt_gpt4",  # Specify the deployment name if needed
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),  # Base URL for the API
    openai_api_type=os.getenv("OPENAI_API_TYPE"),   # API type (usually 'azure' or similar)
    openai_api_version = os.getenv("OPENAI_API_VERSION"),
    temperature=0
)

In [81]:
from langchain import hub
# Get the prompt to use 
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [82]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, combined_tools, prompt)

In [83]:
## Agent executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent, tools=combined_tools, verbose=True)

In [84]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [79]:
agent_executor.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We prov

{'input': 'Tell me about LangSmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith works on its own and does not require the use of LangChain. \n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using pip install -U langsmith for Python or yarn add langsmith for TypeScript.\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n3. Set up your environment by exporting the LANGCHAIN_TRACING_V2 variable as true and the LANGCHAIN_API_KEY variable as your API key. You can also export other variables like OPENAI_API_KEY if necessary.\n4. Log your first trace using the provided methods in LangSmith.\n5. Run your first evaluation by defining a dataset, examples, and evaluators to test your application.\n\nLangSmith also provides tutorials, how-to guides, and reference documen

In [85]:
agent_executor.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We prov

{'input': 'Tell me about LangSmith',
 'output': "LangSmith is a platform for building production-grade Language Learning Model (LLM) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith works independently, and the use of LangChain is not necessary.\n\nHere are the steps to get started with LangSmith:\n\n1. **Install LangSmith**: You can install LangSmith using Python or TypeScript. For Python, use the command `pip install -U langsmith`. For TypeScript, use `yarn add langchain langsmith`.\n\n2. **Create an API key**: To create an API key, head to the Settings page and click 'Create API Key'.\n\n3. **Set up your environment**: You need to set up your environment variables. This includes setting up the LangChain API key and optionally the OpenAI API key.\n\n4. **Log your first trace**: LangSmith provides multiple ways to log traces.\n\n5. **Run your first evaluation**: Evaluation requires a system to test, da

In [86]:
agent_executor.invoke({"input":"Tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Machine learning`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study within artificial intelligence. It is concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. These algorithms improve their performance as they are exposed to more data over time. Machine learning is used in a variety of applications, such as email filtering, detection of network intruders, and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks.

> Finished chain.


{'input': 'Tell me about machine learning',
 'output': 'Machine learning (ML) is a field of study within artificial intelligence. It is concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. These algorithms improve their performance as they are exposed to more data over time. Machine learning is used in a variety of applications, such as email filtering, detection of network intruders, and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks.'}

In [87]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, published on 2016-05-26, studies graphs on lattice points whose edges come from a finite set.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, published on 2016-05-26, studies graphs on lattice points whose edges come from a finite set.'}